In [1]:
import subprocess

import pandas as pd

In [2]:

input_file = "/home/vedant/Desktop/glimpsify/most_info_frame_extractor/experiments/27/ref_timestamps_with_internal_id.csv"


In [3]:
df = pd.read_csv(input_file)
df.head()


,internal_id,video_id,video_name,timestamps
0,mhhqzg,_BvFr9HEiQg,Human Biology Lecture: Chapter 18- Reproductio...,"[24.961, 70.2913, 107.3214, 145.5609, 187.3366..."
1,yaldqc,_e_B8rTl_rU,Pediatric Dentistry | Developmental Disturbanc...,"[32.3223, 48.3223, 95.8223, 258.6495, 273.6494..."
2,znavfo,1gB90u2JOYg,Head & Neck Anatomy | Cranial Nerves | INBDE.mp4,"[19.1948, 290.0761, 361.9277, 607.3729, 684.87..."
3,aqqzpy,2No3_SGYWhk,Prosthodontics | Mandibular Edentulous Anatomy...,"[19.3056, 164.3056, 249.3056, 382.3056, 490.80..."
4,sdcniw,2RkiPyrUxw8,"Periodontics | Local Factors | INBDE, ADAT.mp4","[21.7656, 206.4182, 236.9182, 390.9182, 470.91..."


In [4]:
df.columns

Index(['internal_id', 'video_id', 'video_name', 'timestamps'], dtype='object')

In [ ]:
for internal_id,_,_,timestamps in df.values:
    timestamps = eval(timestamps)
    # timestamps = str(timestamps)
    timestamps = list(map(str, timestamps))
    timestamps = f"{(','.join(timestamps))}"
    if len(internal_id) != 6:
        continue
    try:
        args = [
            "python", "/home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/main.py",
            "--input=local",
            f"--dir={internal_id}",
            "--ocr=tesseract",
            "--extraction=timestamps",
            f'--timestamps',
            timestamps,
            '--ocr_approval=reject_all'
        ]
        print(args)
        subprocess.run(
            args
        )
    except:
        print("Exception occurred for: ", internal_id)

['python', '/home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/main.py', '--input=local', '--dir=mhhqzg', '--ocr=tesseract', '--extraction=timestamps', '--timestamps', '24.961,70.2913,107.3214,145.5609,187.3366,192.8236,205.3429,224.0745,262.083,380.5809,493.1816,515.3795,563.1552,596.7536,632.3407,677.7231,720.4485,741.1738,792.9128,811.8972,1110.519,1188.5773,1236.5091,1307.1052,1357.3302,1402.4769,1551.9644,1605.2989,1659.4705,1715.3827,1769.7411,1792.2949,1874.8464,1894.9419,1902.2482,1919.7455,1937.0351,1976.6208,2061.0764,2075.3232,2156.3919', '--ocr_approval=reject_all']
Created /home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/data/mhhqzg_lpy


Processing Frames: 763it [00:07, 104.04it/s]


Processed 763 frames
Extracted frames to /home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/data/mhhqzg_lpy_extracted_frames
Saved PDF to /home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/data/mhhqzg_lpy.pdf for Human Biology Lecture: Chapter 18- Reproduction and Development.mp4
['python', '/home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/main.py', '--input=local', '--dir=yaldqc', '--ocr=tesseract', '--extraction=timestamps', '--timestamps', '32.3223,48.3223,95.8223,258.6495,273.6494,330.6494,379.6494,447.1494,500.6493,502.6492,549.6492,558.1492,640.6492,752.1492,800.1491,856.6491,867.1491,1053.8505,1091.3505,1186.1302,1290.4699,1420.4699,1500.4699,1540.4699,1598.4699', '--ocr_approval=reject_all']
Created /home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/data/yaldqc_fdi


Processing Frames: 554it [00:05, 97.76it/s] 


Processed 554 frames
Extracted frames to /home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/data/yaldqc_fdi_extracted_frames
Saved PDF to /home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/data/yaldqc_fdi.pdf for Pediatric Dentistry | Developmental Disturbances of Teeth | INBDE, ADAT.mp4
['python', '/home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/main.py', '--input=local', '--dir=znavfo', '--ocr=tesseract', '--extraction=timestamps', '--timestamps', '19.1948,290.0761,361.9277,607.3729,684.8728,721.3727,749.3726,837.3726,961.8725,1114.8384,1136.8383,1303.8977,1336.3976,1509.8772,1602.3772,1628.8771,1652.377', '--ocr_approval=reject_all']
Created /home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/data/znavfo_jcc


...

Traceback (most recent call last):
  File "/home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/main.py", line 119, in <module>
    main()
  File "/home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/main.py", line 109, in main
    directory = input_strategy.proceed()
                ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/input_strategy/local_video_input_strategy.py", line 84, in proceed
    Helper.save_extracted_frames(
  File "/home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/utils/helper.py", line 66, in save_extracted_frames
    cv2.imwrite(frame_output_path, frame)
cv2.error: OpenCV(4.10.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:798: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'



['python', '/home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/main.py', '--input=local', '--dir=mfewuh', '--ocr=tesseract', '--extraction=timestamps', '--timestamps', '28.5358,73.5357,97.5357,169.0357,304.5357,390.5357,447.5357,513.0357,581.5357,610.0356,621.0356,628.0356,676.5356,692.0355,727.5354,813.0354,821.5353,917.0353,963.5353,981.0352,1107.5352,1241.0352,1277.0351,1307.535,1326.5349,1361.5349,1429.0349,1472.0349,1500.0349,1566.5349,1613.5349,1638.0349,1689.5348,1802.0348,1825.0348', '--ocr_approval=reject_all']
Created /home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/data/mfewuh_krc


Processing Frames: 633it [00:05, 109.94it/s]


Processed 633 frames
Extracted frames to /home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/data/mfewuh_krc_extracted_frames
Saved PDF to /home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/data/mfewuh_krc.pdf for Human Biology Lecture: Chapter 7- Cellular Basis of Inheritance (Meiosis).mp4
['python', '/home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/main.py', '--input=local', '--dir=wayvbl', '--ocr=tesseract', '--extraction=timestamps', '--timestamps', '4.5,61.5,204.0,275.5,352.5,364.5,422.0,589.5,661.0,691.5,819.0,834.5,843.5,887.0,992.0,1049.0,1108.0,1171.0,1187.0,1200.5,1230.5,1330.0,1362.5,1439.5,1568.5,1646.5,1786.5,1807.0,1884.5,1971.5,2058.0,2218.0,2327.5,2433.0,2446.5,2489.5,2527.9999,2557.9999,2608.4999,2660.9999,2736.9999,2821.4999,2909.9999,3164.9999,3208.9999,3223.4999,3270.9999,3530.9999,3620.4998,3651.4998,3667.9998,3711.4998,3729.4998,3767.9998,3800.4998,3860.4998,3972.9998,3994.9998,3995.9998', '--ocr_approval=reject_all

Processing Frames: 1351it [00:16, 84.44it/s]


In [30]:

    dir = "/home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/data"
